In [ ]:
!pip install torch
!pip install transformers
!pip install peft
!pip install trl
!pip install -U bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import argparse
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from trl import SFTTrainer, SFTConfig
from transformers import BitsAndBytesConfig
from accelerate import init_empty_weights
from peft import LoraConfig

from peft import PeftModel
import json
import gc
import torch
from torch.utils.data import Dataset

In [ ]:
# 최종 제출 json
output="./result_ensemble.json"

v1 추론

In [ ]:
Category_Array = []
class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.trg = []
        self.label = []

        PROMPT = '''You are a helpful AI assistant. Please answer the user's questions correctly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 바르게 답변해주세요.'''
        answer_dict = {
            "": None,
            "inference_1": 0,
            "inference_2": 1,
            "inference_3": 2
        }

        with open(fname, "r") as f:
            data = json.load(f)

        def make_chat(inp):
            chat = ["[Conversation]"]
            for cvt in inp['conversation']:
                speaker = cvt['speaker']
                utterance = cvt['utterance']
                chat.append(f"화자{speaker}: {utterance} 화자 {speaker} 끝")
            chat = "\n".join(chat)

            question = f"[Question]\n위 대화의 {inp['category']}"
            if (ord(inp['category'][-1]) - ord("가")) % 28 > 0:
                question += "으로"
            else:
                question += "로"

            question += " 올바른 지문은?"

            chat = chat + "\n\n" + question + "\n\n[Option]\n"
            chat += f"A. {inp['inference_1']}\n"
            chat += f"B. {inp['inference_2']}\n"
            chat += f"C. {inp['inference_3']}"

            ### CoT 추가 ###
            cot_guidance = '''
            아래 단계를 따라 위 대화의 질문에 대한 올바른 답을 선택하세요:
            1. 대화의 주요 내용을 파악.
            2. 각 화자의 발언 의도를 파악.
            3. 주어진 선택지(A, B, C)를 분석하여 대화의 맥락과 일치하는지 확인.
            4. 가장 논리적으로 일치하는 지문을 선택지(A, B, C) 중에 하나를 선택.'''

            chat = chat + "\n\n" + cot_guidance

            return chat

        for example in data:
            chat = make_chat(example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]

            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = ""
            if example["output"] == "inference_1":
                target = f"A. {example['input']['inference_1']}{tokenizer.eos_token}"
            elif example["output"] == "inference_2":
                target = f"B. {example['input']['inference_2']}{tokenizer.eos_token}"
            elif example["output"] == "inference_3":
                target = f"C. {example['input']['inference_3']}{tokenizer.eos_token}"

            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))

            self.inp.append(input_ids)
            self.label.append(labels)
            self.trg.append(answer_dict[example["output"]])

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.trg[idx]


class DataCollatorForSupervisedDataset(object):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(ids) for ids in input_ids], batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(lbls) for lbls in labels], batch_first=True, padding_value=-100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:
output_model_path ="./model_path/results_1"

In [ ]:
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

torch.cuda.empty_cache()
gc.collect()

model = AutoModelForCausalLM.from_pretrained(
    "chihoonlee10/T3Q-ko-solar-dpo-v7.0",
    device_map="auto",
    quantization_config = bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("chihoonlee10/T3Q-ko-solar-dpo-v7.0")
tokenizer.pad_token = tokenizer.eos_token

gc.collect()

# LoRA 모델 불러 오기
model = PeftModel.from_pretrained(model, output_model_path)
model.eval()
print("model loaded")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model loaded


In [ ]:
import tqdm
import numpy

dataset = CustomDataset("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", tokenizer)

answer_dict = {
    0: "inference_1",
    1: "inference_2",
    2: "inference_3",
}

with open("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", "r") as f:
    result = json.load(f)

with torch.no_grad():
    prob_array = []
    for idx in tqdm.tqdm(range(len(dataset))):
        inp, _ = dataset[idx]
        outputs = model(
            inp.to("cuda:0").unsqueeze(0)
        )
        logits = outputs.logits[:,-1].flatten()
        probs = (
            torch.nn.functional.softmax(
                torch.tensor(
                    [
                        logits[tokenizer.vocab['A']],
                        logits[tokenizer.vocab['B']],
                        logits[tokenizer.vocab['C']],
                    ]
                ),
                dim=0,
            )
            .detach()
            .cpu()
            .to(torch.float32)
            .numpy()
        )

        #확률값 저장 (앙상블 위해)
        prob_array.append(probs)

        torch.cuda.empty_cache()
        gc.collect()

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
100%|██████████| 605/605 [12:18<00:00,  1.22s/it]


In [ ]:
rounded_arrays = [np.round(arr, 3) for arr in prob_array]

v2 추론

In [ ]:
output_model_path ="./model_path/results_2"

In [ ]:
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

torch.cuda.empty_cache()
gc.collect()

model = AutoModelForCausalLM.from_pretrained(
    "chihoonlee10/T3Q-ko-solar-dpo-v7.0",
    device_map="auto",
    quantization_config = bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("chihoonlee10/T3Q-ko-solar-dpo-v7.0")
tokenizer.pad_token = tokenizer.eos_token

gc.collect()

# LoRA 모델 불러 오기
model = PeftModel.from_pretrained(model, output_model_path)
model.eval()
print("model loaded")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

model loaded


In [ ]:
## 다른 chat prompt ##
class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.trg = []
        self.label = []

        PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
        answer_dict = {
            "": None,
            "inference_1": 0,
            "inference_2": 1,
            "inference_3": 2
        }

        with open(fname, "r") as f:
            data = json.load(f)

        def make_chat(inp):
            chat = ["[Conversation]"]
            for cvt in inp['conversation']:
                speaker = cvt['speaker']
                utterance = cvt['utterance']
                utterance_id = cvt['utterance_id']
                chat.append(f"화자{speaker} [{utterance_id}]: {utterance}")
            chat = "\n".join(chat)

            utterance_array = []
            for utterance_id in inp['reference_id']:
                utterance_array.append(utterance_id)  #conversation utterance_id 가져오기

            question = f"[Question]\n위 대화의 {utterance_array} 참고해서 {inp['category']}"
            if (ord(inp['category'][-1]) - ord("가")) % 28 > 0:
                question += "으로"
            else:
                question += "로"

            question += " 올바른 지문은?"

            chat = chat + "\n\n" + question + "\n\n[Option]\n"
            chat += f"A. {inp['inference_1']}\n"
            chat += f"B. {inp['inference_2']}\n"
            chat += f"C. {inp['inference_3']}"

            return chat

        for example in data:
            chat = make_chat(example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]
            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = ""
            if example["output"] == "inference_1":
                target = f"A. {example['input']['inference_1']}{tokenizer.eos_token}"
            elif example["output"] == "inference_2":
                target = f"B. {example['input']['inference_2']}{tokenizer.eos_token}"
            elif example["output"] == "inference_3":
                target = f"C. {example['input']['inference_3']}{tokenizer.eos_token}"

            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))

            self.inp.append(input_ids)
            self.label.append(labels)
            self.trg.append(answer_dict[example["output"]])

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.trg[idx]


In [ ]:
import tqdm
import numpy

dataset = CustomDataset("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", tokenizer)

answer_dict = {
    0: "inference_1",
    1: "inference_2",
    2: "inference_3",
}

with open("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", "r") as f:
    result_2 = json.load(f)

with torch.no_grad():
    prob_array_2 = []
    for idx in tqdm.tqdm(range(len(dataset))):
        inp, _ = dataset[idx]
        outputs = model(
            inp.to("cuda:0").unsqueeze(0)
        )
        logits = outputs.logits[:,-1].flatten()
        probs = (
            torch.nn.functional.softmax(
                torch.tensor(
                    [
                        logits[tokenizer.vocab['A']],
                        logits[tokenizer.vocab['B']],
                        logits[tokenizer.vocab['C']],
                    ]
                ),
                dim=0,
            )
            .detach()
            .cpu()
            .to(torch.float32)
            .numpy()
        )

        #확률값 저장 (앙상블 위해)
        prob_array_2.append(probs)

        torch.cuda.empty_cache()
        gc.collect()


100%|██████████| 605/605 [13:16<00:00,  1.32s/it]


In [ ]:
rounded_arrays_2 = [np.round(arr, 3) for arr in prob_array_2]

v3 추론

In [ ]:
output_model_path ="./model_path/results_3"

In [ ]:
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

# To prevent GPU memory overflow in Mixtral8x7b
config = AutoConfig.from_pretrained('rtzr/ko-gemma-2-9b-it')
config.gradient_checkpointing = True

torch.cuda.empty_cache()
gc.collect()

model = AutoModelForCausalLM.from_pretrained(
    "rtzr/ko-gemma-2-9b-it",
    device_map="auto",
    quantization_config = bnb_config,
    config=config
)

tokenizer = AutoTokenizer.from_pretrained("rtzr/ko-gemma-2-9b-it")
tokenizer.pad_token = tokenizer.eos_token

gc.collect()

# LoRA 모델 불러 오기
model = PeftModel.from_pretrained(model, output_model_path)
model.eval()
print("model loaded")

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model loaded


In [ ]:
## 다른 chat prompt ##
class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.trg = []
        self.label = []

        PROMPT = '''You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''
        answer_dict = {
            "": None,
            "inference_1": 0,
            "inference_2": 1,
            "inference_3": 2
        }

        with open(fname, "r") as f:
            data = json.load(f)

        def make_chat(inp):
            chat = ["[Conversation]"]
            for cvt in inp['conversation']:
                speaker = cvt['speaker']
                utterance = cvt['utterance']
                utterance_id = cvt['utterance_id']
                chat.append(f"화자{speaker} [{utterance_id}]: {utterance}")
            chat = "\n".join(chat)

            utterance_array = []
            for utterance_id in inp['reference_id']:
                utterance_array.append(utterance_id)  #conversation utterance_id 가져오기

            question = f"[Question]\n위 대화의 {utterance_array} 참고해서 {inp['category']}"
            if (ord(inp['category'][-1]) - ord("가")) % 28 > 0:
                question += "으로"
            else:
                question += "로"

            question += " 올바른 지문은?"

            chat = chat + "\n\n" + question + "\n\n[Option]\n"
            chat += f"A. {inp['inference_1']}\n"
            chat += f"B. {inp['inference_2']}\n"
            chat += f"C. {inp['inference_3']}"

            return chat

        for example in data:
            chat = make_chat(example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]
            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = ""
            if example["output"] == "inference_1":
                target = f"A. {example['input']['inference_1']}{tokenizer.eos_token}"
            elif example["output"] == "inference_2":
                target = f"B. {example['input']['inference_2']}{tokenizer.eos_token}"
            elif example["output"] == "inference_3":
                target = f"C. {example['input']['inference_3']}{tokenizer.eos_token}"


            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))

            self.inp.append(input_ids)
            self.label.append(labels)
            self.trg.append(answer_dict[example["output"]])

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.trg[idx]


In [ ]:
import tqdm
import numpy

dataset = CustomDataset("/dataset/대화맥락추론_데이터/대화맥락추론_test.json", tokenizer)

answer_dict = {
    0: "inference_1",
    1: "inference_2",
    2: "inference_3",
}

with open("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", "r") as f:
    result_3 = json.load(f)

with torch.no_grad():
    prob_array_3 = []
    for idx in tqdm.tqdm(range(len(dataset))):
        inp, _ = dataset[idx]
        outputs = model(
            inp.to("cuda:0").unsqueeze(0)
        )
        logits = outputs.logits[:,-1].flatten()
        probs = (
            torch.nn.functional.softmax(
                torch.tensor(
                    [
                        logits[tokenizer.vocab['A']],
                        logits[tokenizer.vocab['B']],
                        logits[tokenizer.vocab['C']],
                    ]
                ),
                dim=0,
            )
            .detach()
            .cpu()
            .to(torch.float32)
            .numpy()
        )

        #확률값 저장 (앙상블 위해)
        prob_array_3.append(probs)

        torch.cuda.empty_cache()
        gc.collect()


100%|██████████| 605/605 [13:12<00:00,  1.31s/it]


In [ ]:
rounded_arrays_3 = [np.round(arr, 3) for arr in prob_array_3]

v4 추론

In [ ]:
output_model_path ="./model_path/results_4"

In [ ]:
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

# To prevent GPU memory overflow in Mixtral8x7b
config = AutoConfig.from_pretrained('chihoonlee10/T3Q-ko-solar-dpo-v7.0')
config.gradient_checkpointing = True

torch.cuda.empty_cache()
gc.collect()

model = AutoModelForCausalLM.from_pretrained(
    "chihoonlee10/T3Q-ko-solar-dpo-v7.0",
    device_map="auto",
    quantization_config = bnb_config,
    config=config
)

tokenizer = AutoTokenizer.from_pretrained("chihoonlee10/T3Q-ko-solar-dpo-v7.0")
tokenizer.pad_token = tokenizer.eos_token

gc.collect()

# LoRA 모델 불러 오기
model = PeftModel.from_pretrained(model, output_model_path)
model.eval()
print("model loaded")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

model loaded


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.trg = []
        self.label = []

        PROMPT = '''You are a helpful AI assistant. Please answer the user's questions correctly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 올바르게 답변해주세요.
        아래 단계를 따라 주어진 대화의 맥락을 이해하고 질문에 대한 올바른 답을 선택하세요:
        1. 대화의 주요 내용을 파악.
        2. 각 화자의 발언 의도를 파악.
        3. 주어진 선택지(A, B, C)를 분석하여 대화의 맥락과 일치하는지 확인.
        4. 가장 논리적으로 일치하는 지문 하나를 선택.'''
        answer_dict = {
            "": None,
            "inference_1": 0,
            "inference_2": 1,
            "inference_3": 2
        }

        with open(fname, "r") as f:
            data = json.load(f)

        def make_add_info(inp):
            question =''

            if  inp['category'] == '원인':
                question += '(대상 발화의 사건을 유발하는 사건)'
            elif inp['category'] == '후행 사건':
                question += '(대상 발화 이후에 일어날 수 있는 사건)'
            elif inp['category'] == '전제 조건':
                question += '(대상 발화의 사건을 가능하게 하는 상태 혹은 사건)'
            elif inp['category'] == '내적 동기':
                question += '(대상 발화를 일으키는 ‘화자’의 감정이나 기본 욕구)'
            elif inp['category'] == '감정 반응':
                question += '(대상 발화 사건에 대해 ‘청자’가 보일 수 있는 감정 반응)'

            return question

        def make_chat(inp):
            chat = ["[Conversation]"]
            for cvt in inp['conversation']:
                speaker = cvt['speaker']
                utterance = cvt['utterance']
                chat.append(f"화자{speaker}: {utterance} 화자 {speaker} 끝")
            chat = "\n".join(chat)

            question = f"[Question]\n위 대화의 {inp['category']}"
            # 유형 정보 추가
            question += make_add_info(inp)

            if (ord(inp['category'][-1]) - ord("가")) % 28 > 0:
                question += "으로"
            else:
                question += "로"

            question += " 올바른 지문은?"

            chat = chat + "\n\n" + question + "\n\n[Option]\n"
            chat += f"A. {inp['inference_1']}\n"
            chat += f"B. {inp['inference_2']}\n"
            chat += f"C. {inp['inference_3']}"

            return chat

        for example in data:
            chat = make_chat(example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]
            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = ""
            if example["output"] == "inference_1":
                target = f"A. {example['input']['inference_1']}{tokenizer.eos_token}"
            elif example["output"] == "inference_2":
                target = f"B. {example['input']['inference_2']}{tokenizer.eos_token}"
            elif example["output"] == "inference_3":
                target = f"C. {example['input']['inference_3']}{tokenizer.eos_token}"

            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))

            self.inp.append(input_ids)
            self.label.append(labels)
            self.trg.append(answer_dict[example["output"]])

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.trg[idx]


In [ ]:
import tqdm
import numpy

dataset = CustomDataset("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", tokenizer)

answer_dict = {
    0: "inference_1",
    1: "inference_2",
    2: "inference_3",
}

with open("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", "r") as f:
    result_4 = json.load(f)

with torch.no_grad():
    prob_array_4 = []
    for idx in tqdm.tqdm(range(len(dataset))):
        inp, _ = dataset[idx]
        outputs = model(
            inp.to("cuda:0").unsqueeze(0)
        )
        logits = outputs.logits[:,-1].flatten()
        probs = (
            torch.nn.functional.softmax(
                torch.tensor(
                    [
                        logits[tokenizer.vocab['A']],
                        logits[tokenizer.vocab['B']],
                        logits[tokenizer.vocab['C']],
                    ]
                ),
                dim=0,
            )
            .detach()
            .cpu()
            .to(torch.float32)
            .numpy()
        )

        #확률값 저장 (앙상블 위해)
        prob_array_4.append(probs)

        torch.cuda.empty_cache()
        gc.collect()


100%|██████████| 605/605 [15:29<00:00,  1.54s/it]


In [ ]:
rounded_arrays_4 = [np.round(arr, 3) for arr in prob_array_4]

csv 파일 생성

In [ ]:
import csv
csv_file = './rounded_arrays.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rounded_arrays)

In [ ]:
csv_file = './rounded_arrays_2.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rounded_arrays_2)

In [ ]:
csv_file = './rounded_arrays_3.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rounded_arrays_3)

In [ ]:
csv_file = './rounded_arrays_4.csv'

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rounded_arrays_4)

앙상블 진행

In [ ]:
import pandas as pd

# CSV 파일들을 읽어옵니다 (컬럼 이름이 없으므로 header=None 사용)
df1 = pd.read_csv('./rounded_arrays.csv', header=None)
df2 = pd.read_csv('./rounded_arrays_2.csv', header=None)
df3 = pd.read_csv('./rounded_arrays_3.csv', header=None)
df4 = pd.read_csv('./rounded_arrays_4.csv', header=None)

# 두 DataFrame을 더합니다
# result_df = (df1 * 0.25) + (df2 * 0.25) + (df3 * 0.25) + (df4 * 0.25) # 리더 보드 기준 96.0점 기록
result_df = (df1 * 0.25) + (df2 * 0.30) + (df3 * 0.30) + (df4 * 0.15)

# 결과를 새로운 CSV 파일로 저장합니다
result_df.to_csv('./ensemble_result.csv', header=False, index=False)

print("CSV 파일들이 성공적으로 더해졌습니다.")

CSV 파일들이 성공적으로 더해졌습니다.


In [ ]:
import tqdm
import numpy

dataset = CustomDataset("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", tokenizer)

answer_dict = {
    0: "inference_1",
    1: "inference_2",
    2: "inference_3",
}

with open("./dataset/대화맥락추론_데이터/대화맥락추론_test.json", "r") as f:
    result_t = json.load(f)

# 확률값 활용한 데이터 프레임을 리스트로 변환
probs_ensemble = result_df.values.tolist()

for idx in tqdm.tqdm(range(len(dataset))):

    # 확률값 활용
    probs = probs_ensemble[idx]

    result_t[idx]["output"] = answer_dict[numpy.argmax(probs)]


with open(output, "w", encoding="utf-8") as f:
    f.write(json.dumps(result_t, ensure_ascii=False, indent=4))

100%|██████████| 605/605 [00:00<00:00, 138983.13it/s]
